In [5]:


import numpy as np
from time import time as timer
from time import time_ns as timer_ns
from timeit import default_timer as default_timer
def checktick(fn):
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = fn() # get timestamp from timer
        t2 = fn() # get timestamp from timer
        while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
            t2 = fn() # get timestamp from timer
        t1 = t2 # this is outside the loop
        timesfound[i] = t1 # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound) # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta

if __name__ == "__main__":
    timer_gran=checktick(timer)
    timer_ns_gran=checktick(timer_ns)/pow(10,9)
    default_timer_gran=checktick(default_timer)
    print(f"Granularity for timer_ns(): {timer_gran}")
    print(f"Granularity for timer(): {timer_ns_gran}")
    print(f"Granularity for default_timer(): {default_timer_gran}")


Granularity for timer_ns(): 7.152557373046875e-07
Granularity for timer(): 7.68e-07
Granularity for default_timer(): 2.4959444999694824e-07


In [7]:
"""Julia set generator without optional PIL-based image drawing"""
from timeit import default_timer as time
from functools import wraps

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# decorator to time
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time()
        result = fn(*args, **kwargs)
        t2 = time()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time

@timefn
def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    assert sum(output) == 33219980

@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300) 

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 2.9410407091490924 seconds
calculate_z_serial_purepython took 2.9411374162882566 seconds
@timefn: calc_pure_python took 3.1092668338678777 seconds


A screenshot of snakeviz diagrams of the notiming.stats file
![title](JuliaSnakeViz.png)

In [20]:
! python3 -m line_profiler JuliaSet.py.lprof

Timer unit: 1e-06 s

Total time: 25.7086 s
File: JuliaSet.py
Function: calc_pure_python at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                           #@profile
    24                                           def calc_pure_python(desired_width, max_iterations):
    25                                               """Create a list of complex coordinates (zs) and complex parameters (cs),
    26                                               build Julia set"""
    27         1          1.0      1.0      0.0      x_step = (x2 - x1) / desired_width
    28         1          0.0      0.0      0.0      y_step = (y1 - y2) / desired_width
    29         1          0.0      0.0      0.0      x = []
    30         1          0.0      0.0      0.0      y = []
    31         1          0.0      0.0      0.0      ycoord = y2
    32      1001        107.0      0.1      0.0      while ycoord > y1:
    33      1000         76.0      0.1      

The overhead, while running locally on a M1 Macbook Pro, for cProfile is roughly 3 seconds increasing the runtime 3 seconds to 6. While running kernprof in order to use line_profiler adds 22.7 seconds to the runtime.

![title](mprofPlot.png)

The overhead, while running locally on a M1 Macbook Pro, for memory_profiler with a 100x100 grid is 14.1 seconds from 0.04, or a 350x increase. While mprof on a 1000x1000 grid had a negligble overhead.

A screenshot of snakeviz diagrams of the diffSats.stats file
![title](DiffusionSnakeViz.png)

![title](DiffusionPlot.png)



In [21]:
from time import perf_counter as time
import subprocess
import psutil
import sys
import threading 
import numpy as np
from functools import wraps

def collectUtil(fn):
    @wraps(fn)
    def measure(*args, **kwargs):
        def collect(event, args):
                f=open("test.dat", "w")
                measurements=np.array([0]*psutil.cpu_count(logical=False))
                for i in measurements:
                    f.write(str(i) + " ")
                f.write(f"{time()} \n")
                while not event.is_set():
                    perc=psutil.cpu_percent(interval=0.25, percpu=True)
                    for i in perc:
                        f.write(str(i) + " ")
                    f.write(f"{time()} \n")
                f.close()
                return
        stopEvent=threading.Event()
        t=threading.Thread(target=collect, args=(stopEvent, False))
        t.start()
        result=fn(*args, **kwargs)
        stopEvent.set()
        t.join()
        subprocess.Popen([sys.executable, "PlotCPU.py", "test.dat"])
        return result
    return measure



In [25]:
import matplotlib.pyplot as plt
import numpy as np
import sys
def doPlot(filename):
    data=np.loadtxt(filename)
    shape=np.shape(data)
    startval=data[:,-1][0]
    data[:,-1]=data[:,-1]-startval
    print("\nCore usage in percentage")
    for i in range(shape[1]-1):
        string="Core " + str(i+1)
        print(f"{string:<11}", end='', flush=True)
    print(f"{"time":<11}")

    def mapFunc(val):
        return f"{val:<10.5}"
    for i in range(shape[0]):
        res=map(mapFunc, data[i,:])
        print(*res)
    plt.figure(figsize=(10,6))
    for i in range(shape[1]-1):
        plt.plot(data[:,-1],data[:,i], label=f"Core {i+1}")
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid()
    plt.axis([None, None, 0, 100])
    plt.show()


   
    #Arg is .dat file
if __name__ == "__main__":
    pass
    #doPlot(sys.argv[1]) Uncomment when actually used

![title](JuliaCpu.png)

![title](DiffusionCpu.png)